In [ ]:

pip install torch torchvision torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910460 sha256=269d409db90cea5fe3e1eea4ff885269e693d2e81ca8dc97cd1c94b846143101
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [ ]:
!pip install torch-geometric


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import DataLoader, Data
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv  # Use GCNConv from torch_geometric.nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize
from torchvision.transforms import Compose
from torch.utils.data.dataset import random_split

# Define GNN model using GCNConv
class GNNModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)





# Load Cora dataset
dataset = Planetoid(root='/content/data/Planetoid', name='Cora', transform=NormalizeFeatures())

# Set hyperparameters
num_features = dataset.num_features
hidden_size = 64
num_classes = dataset.num_classes
learning_rate = 0.01
num_epochs = 100

# Create GNN model and optimizer
model = GNNModel(num_features, hidden_size, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Prepare the data loader for training
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


# Calculate TP, FP, TN, FN
model.eval()
TP = 0
FP = 0
TN = 0
FN = 0

positive_class = 1  # Replace with the actual positive class label
negative_class = 0  # Replace with the actual negative class label

with torch.no_grad():
    for data in train_loader:
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += data.y.size(0)
        correct += (predicted == data.y).sum().item()

        # Calculate TP, FP, TN, FN for each batch
        TP += ((predicted == data.y) & (data.y == positive_class)).sum().item()
        FP += ((predicted != data.y) & (data.y == negative_class)).sum().item()
        TN += ((predicted == data.y) & (data.y == negative_class)).sum().item()
        FN += ((predicted != data.y) & (data.y == positive_class)).sum().item()

# Calculate accuracy, precision, recall, specificity and F1 score
accuracy = 100 * correct / total
precision = TP / (TP + FP)
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
f1_score = 2 * precision * recall / (precision + recall)

print(f'Test Accuracy: {accuracy:.2f}%')
print(f'Test Precision: {precision:.2f}')
print(f'Sensitivity: {recall:.2f}')
print(f'Specificity: {specificity:.2f}')
print(f'F1 Score: {f1_score:.2f}')

# Load images from Google Drive folder for Node Classification
data_transform = Compose([Resize((224, 224)), ToTensor()])
data_folder = '/content/drive/MyDrive/ahsraf.v1i.yolov7pytorch'
image_dataset = ImageFolder(root=data_folder, transform=data_transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(image_dataset))
test_size = len(image_dataset) - train_size
train_dataset, test_dataset = random_split(image_dataset, [train_size, test_size])

# Create data loaders for train and test sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Continue with your Node Classification with GNN using the train_loader and test_loader
# ...

# Remember to adjust the paths and other hyperparameters according to your dataset and use case.

Epoch [1/100], Loss: 1.9430
Epoch [2/100], Loss: 1.9291
Epoch [3/100], Loss: 1.9096
Epoch [4/100], Loss: 1.8828
Epoch [5/100], Loss: 1.8532
Epoch [6/100], Loss: 1.8276
Epoch [7/100], Loss: 1.8034
Epoch [8/100], Loss: 1.7992
Epoch [9/100], Loss: 1.7900
Epoch [10/100], Loss: 1.7609
Epoch [11/100], Loss: 1.7343
Epoch [12/100], Loss: 1.7099
Epoch [13/100], Loss: 1.6787
Epoch [14/100], Loss: 1.6509
Epoch [15/100], Loss: 1.6073
Epoch [16/100], Loss: 1.5670
Epoch [17/100], Loss: 1.5207
Epoch [18/100], Loss: 1.4705
Epoch [19/100], Loss: 1.4162
Epoch [20/100], Loss: 1.3619
Epoch [21/100], Loss: 1.3050
Epoch [22/100], Loss: 1.2508
Epoch [23/100], Loss: 1.2022
Epoch [24/100], Loss: 1.1535
Epoch [25/100], Loss: 1.1105
Epoch [26/100], Loss: 1.0501
Epoch [27/100], Loss: 1.0088
Epoch [28/100], Loss: 0.9660
Epoch [29/100], Loss: 0.9223
Epoch [30/100], Loss: 0.8949
Epoch [31/100], Loss: 0.8554
Epoch [32/100], Loss: 0.8206
Epoch [33/100], Loss: 0.7967
Epoch [34/100], Loss: 0.7688
Epoch [35/100], Loss: 0